In [51]:
#Carregando os pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')


In [52]:
#Configurando o diretório e carregando o dataset
os.chdir('/content/drive/MyDrive/Bases de Dados para Indicadores EUROFOUND')
life_exp_states_br = pd.read_csv('ADH_BASE_RADAR_2012-2021.csv')

In [53]:
#analisando o dataset
life_exp_states_br.head()

,ANO,AGREGACAO,CODIGO,NOME,IDHM,IDHM_L,IDHM_E,IDHM_R,I_ESCOLARIDADE,I_FREQ_PROP,...,POP6A17,POP11A13,POP15A17,POP15M,POP18A20,POP18A24,POP18M,POP25M,POP65M,POPOCUP18M
0,2012,BRASIL,NaN,NaN,0.746,0.825,0.687,0.732,0.606,0.731,...,"39,808,859","10,304,212","10,609,288","150,994,845","10,076,087","23,924,348","140,385,557","116,461,209","15,249,133","84,277,753"
1,2013,BRASIL,NaN,NaN,0.753,0.830,0.698,0.737,0.615,0.744,...,"39,055,262","9,986,737","10,521,249","153,338,252","10,336,071","24,034,903","142,817,003","118,782,100","15,820,311","86,108,710"
2,2014,BRASIL,NaN,NaN,0.762,0.835,0.715,0.741,0.630,0.761,...,"38,765,780","9,687,655","10,446,675","155,356,978","10,224,771","23,840,076","144,910,303","121,070,227","16,425,141","87,308,460"
3,2015,BRASIL,NaN,NaN,0.765,0.840,0.724,0.736,0.642,0.769,...,"38,189,476","9,314,133","10,401,307","157,576,293","10,394,554","23,851,445","147,174,986","123,323,541","17,060,247","87,756,538"
4,2016,BRASIL,NaN,NaN,0.771,0.845,0.740,0.734,0.657,0.786,...,"37,694,949","9,288,033","10,375,148","159,692,542","10,242,815","23,779,687","149,317,393","125,537,706","17,728,449","87,110,394"


In [54]:
life_exp_states_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Data columns (total 83 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                490 non-null    int64  
 1   AGREGACAO          490 non-null    object 
 2   CODIGO             480 non-null    float64
 3   NOME               480 non-null    object 
 4   IDHM               490 non-null    float64
 5   IDHM_L             490 non-null    float64
 6   IDHM_E             490 non-null    float64
 7   IDHM_R             490 non-null    float64
 8   I_ESCOLARIDADE     490 non-null    float64
 9   I_FREQ_PROP        490 non-null    float64
 10  IDHMAD             490 non-null    float64
 11  IDHMAD_L           490 non-null    float64
 12  IDHMAD_E           490 non-null    float64
 13  IDHMAD_R           490 non-null    float64
 14  ATK_ESPVIDA        490 non-null    float64
 15  ATK_ANOSEST        490 non-null    float64
 16  ATK_RDPC           490 non

In [55]:
#Selecionando as colunas que precisamos
life_exp_states_br = life_exp_states_br[['ANO', 'AGREGACAO', 'CODIGO', 'ESPVIDA']]
life_exp_states_br.head()

,ANO,AGREGACAO,CODIGO,ESPVIDA
0,2012,BRASIL,NaN,74.48
1,2013,BRASIL,NaN,74.80
2,2014,BRASIL,NaN,75.11
3,2015,BRASIL,NaN,75.40
4,2016,BRASIL,NaN,75.68


In [56]:
#Excluindo linhas NaN
life_exp_states_br = life_exp_states_br.dropna()
life_exp_states_br

,ANO,AGREGACAO,CODIGO,ESPVIDA
10,2012,UF,11.0,70.50
11,2012,UF,12.0,72.47
12,2012,UF,13.0,70.85
13,2012,UF,14.0,70.10
14,2012,UF,15.0,71.23
...,...,...,...,...
485,2021,RM_RIDE,41.0,76.15
486,2021,RM_RIDE,42.0,76.71
487,2021,RM_RIDE,43.0,75.05
488,2021,RM_RIDE,51.0,70.73


In [57]:
#Transformando coluna 'CODIGO' em int
life_exp_states_br['CODIGO'] = life_exp_states_br['CODIGO'].astype(int)
life_exp_states_br.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, 10 to 489
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ANO        480 non-null    int64  
 1   AGREGACAO  480 non-null    object 
 2   CODIGO     480 non-null    int64  
 3   ESPVIDA    480 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 18.8+ KB


In [58]:
#Excluindo as linhas das regiões metropolitanas
life_exp_states_br['AGREGACAO']= life_exp_states_br['AGREGACAO'].astype(str)
padrao = 'RM_RIDE'
mascara = life_exp_states_br['AGREGACAO'].str.contains(padrao)
life_exp_states_br = life_exp_states_br.drop(life_exp_states_br[mascara].index)
life_exp_states_br

,ANO,AGREGACAO,CODIGO,ESPVIDA
10,2012,UF,11,70.50
11,2012,UF,12,72.47
12,2012,UF,13,70.85
13,2012,UF,14,70.10
14,2012,UF,15,71.23
...,...,...,...,...
275,2021,UF,43,72.84
276,2021,UF,50,70.03
277,2021,UF,51,68.77
278,2021,UF,52,68.28


In [59]:
#Excluindo a coluna AGREGACAO
life_exp_states_br = life_exp_states_br.drop('AGREGACAO', axis=1)
life_exp_states_br

,ANO,CODIGO,ESPVIDA
10,2012,11,70.50
11,2012,12,72.47
12,2012,13,70.85
13,2012,14,70.10
14,2012,15,71.23
...,...,...,...
275,2021,43,72.84
276,2021,50,70.03
277,2021,51,68.77
278,2021,52,68.28


In [60]:
#Pivotando o dataset para formato Tidy
pivot_life_exp_states_br = life_exp_states_br.pivot(index='ANO', columns='CODIGO', values='ESPVIDA')
pivot_life_exp_states_br

CODIGO,11,12,13,14,15,16,17,21,22,23,...,32,33,35,41,42,43,50,51,52,53
ANO,,,,,,,,,,,,,,,,,,,,,
2012,70.50,72.47,70.85,70.10,71.23,72.77,72.11,69.34,70.19,72.78,...,76.78,74.90,76.80,75.76,77.70,76.62,74.35,73.16,73.41,76.87
2013,70.71,72.84,71.11,70.44,71.44,73.07,72.42,69.65,70.38,73.03,...,77.17,75.23,77.15,76.09,78.07,76.91,74.65,73.43,73.58,77.17
2014,70.91,73.21,71.36,70.77,71.64,73.37,72.71,69.96,70.57,73.26,...,77.54,75.56,77.47,76.40,78.42,77.21,74.94,73.69,73.75,77.47
2015,71.11,73.56,71.61,71.10,71.84,73.65,73.01,70.25,70.76,73.49,...,77.89,75.88,77.78,76.72,78.76,77.49,75.22,73.95,73.92,77.75
2016,71.30,73.89,71.85,71.41,72.03,73.92,73.29,70.54,70.94,73.72,...,78.22,76.18,78.07,77.02,79.09,77.77,75.49,74.20,74.09,78.02
2017,71.50,74.20,72.08,71.71,72.22,74.18,73.56,70.83,71.12,73.94,...,78.53,76.47,78.35,77.32,79.40,78.04,75.74,74.44,74.26,78.27
2018,71.69,74.51,72.31,72.01,72.41,74.42,73.81,71.11,71.30,74.15,...,78.83,76.76,78.63,77.61,79.70,78.30,76.00,74.68,74.42,78.52
2019,71.88,74.78,72.53,72.29,72.59,74.66,74.05,71.39,71.47,74.37,...,79.12,77.04,78.87,77.89,79.98,78.55,76.24,74.91,74.59,78.76
2020,71.20,74.47,71.99,72.00,72.02,73.91,74.02,70.45,71.06,74.29,...,78.68,73.24,75.63,74.20,76.47,74.88,72.34,71.16,70.76,75.17


In [61]:
#Formatando o dataset
pivot_life_exp_states_br.reset_index(inplace=True)
pivot_life_exp_states_br.rename(columns={'index': 'ANO'}, inplace=True)

In [62]:
life_exp_states_br = pivot_life_exp_states_br.rename(columns={'ANO': 'time'})
life_exp_states_br

CODIGO,time,11,12,13,14,15,16,17,21,22,...,32,33,35,41,42,43,50,51,52,53
0,2012,70.50,72.47,70.85,70.10,71.23,72.77,72.11,69.34,70.19,...,76.78,74.90,76.80,75.76,77.70,76.62,74.35,73.16,73.41,76.87
1,2013,70.71,72.84,71.11,70.44,71.44,73.07,72.42,69.65,70.38,...,77.17,75.23,77.15,76.09,78.07,76.91,74.65,73.43,73.58,77.17
2,2014,70.91,73.21,71.36,70.77,71.64,73.37,72.71,69.96,70.57,...,77.54,75.56,77.47,76.40,78.42,77.21,74.94,73.69,73.75,77.47
3,2015,71.11,73.56,71.61,71.10,71.84,73.65,73.01,70.25,70.76,...,77.89,75.88,77.78,76.72,78.76,77.49,75.22,73.95,73.92,77.75
4,2016,71.30,73.89,71.85,71.41,72.03,73.92,73.29,70.54,70.94,...,78.22,76.18,78.07,77.02,79.09,77.77,75.49,74.20,74.09,78.02
5,2017,71.50,74.20,72.08,71.71,72.22,74.18,73.56,70.83,71.12,...,78.53,76.47,78.35,77.32,79.40,78.04,75.74,74.44,74.26,78.27
6,2018,71.69,74.51,72.31,72.01,72.41,74.42,73.81,71.11,71.30,...,78.83,76.76,78.63,77.61,79.70,78.30,76.00,74.68,74.42,78.52
7,2019,71.88,74.78,72.53,72.29,72.59,74.66,74.05,71.39,71.47,...,79.12,77.04,78.87,77.89,79.98,78.55,76.24,74.91,74.59,78.76
8,2020,71.20,74.47,71.99,72.00,72.02,73.91,74.02,70.45,71.06,...,78.68,73.24,75.63,74.20,76.47,74.88,72.34,71.16,70.76,75.17
9,2021,68.86,72.26,69.61,69.71,69.62,71.70,71.74,67.90,68.57,...,76.86,71.12,73.61,72.11,74.60,72.84,70.03,68.77,68.28,73.16


In [67]:
life_exp_states_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    10 non-null     int64  
 1   11      10 non-null     float64
 2   12      10 non-null     float64
 3   13      10 non-null     float64
 4   14      10 non-null     float64
 5   15      10 non-null     float64
 6   16      10 non-null     float64
 7   17      10 non-null     float64
 8   21      10 non-null     float64
 9   22      10 non-null     float64
 10  23      10 non-null     float64
 11  24      10 non-null     float64
 12  25      10 non-null     float64
 13  26      10 non-null     float64
 14  27      10 non-null     float64
 15  28      10 non-null     float64
 16  29      10 non-null     float64
 17  31      10 non-null     float64
 18  32      10 non-null     float64
 19  33      10 non-null     float64
 20  35      10 non-null     float64
 21  41      10 non-null     float64
 22  42   

In [66]:
life_exp_states_br.to_csv('/content/drive/MyDrive/Bases de Dados para Indicadores EUROFOUND/life_exp_states_br.csv', encoding='UTF-8', index=False)